In [ ]:
import nltk
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

True

In [ ]:
pip install xgboost

In [ ]:
import sys  
!{sys.executable} -m pip install contractions

     |████████████████████████████████| 317kB 15.1MB/s 
     |████████████████████████████████| 245kB 21.9MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81696 sha256=8453e97323e4af00f3a2e47392888361122516035aa4ebd42869a7e246438062
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [ ]:
!pip install -U scikit-learn

     |████████████████████████████████| 6.8MB 4.2MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [ ]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
import re
import contractions
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [ ]:
def text_preprocess(sent):
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer() 
    
    # convert all words to lowercase
    sent = sent.lower()
    
    # expand contractions
    expanded_words = []
    for word in sent.split():
        expanded_words.append(contractions.fix(word))
    sent = ' '.join(expanded_words) 
    
    # remove html tags
    sent = re.sub('{html}', "", sent)
    
    # remove http links and web site url
    sent = re.sub(r"http\S+", "", sent)
    sent = re.sub(r"www\S+", "", sent)
    
    # remove numbers
    sent = re.sub('[0-9]+', '', sent)
    
    # remove words utc and image
    sent = sent.replace('utc', '')
    
    # remove words with .jpg
    sent = re.sub(r"[a-zA-Z]*[0-9]*.jpg", '', sent)
    
    # remove emails
    sent = re.sub(r"\S*@\S*\s?", '', sent)
    
    # tokenization
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sent)
    
    # remove stop words
    filtered_words = [w for w in tokens if w not in stopwords.words('english')]
    
    stem_words = [stemmer.stem(w) for w in filtered_words]
    lemma_words = [lemmatizer.lemmatize(w) for w in stem_words]
    
    return " ".join(lemma_words)

In [ ]:
text = pd.read_csv('train.csv')
text.dropna(inplace=True)

In [ ]:
text['comment_text'] = text['comment_text'].map(lambda com : text_preprocess(com))

In [ ]:
text.to_csv("preprocessed_text.csv", index=False, header=True)

In [ ]:
def remove_long(sent):
    sentence = []
    for word in sent.split():
        if len(word)>15:
            continue
        else:
            sentence.append(word)
    return ' '.join(sentence)

In [ ]:
com = pd.read_csv('preprocessed_text.csv')
com.dropna(inplace=True)
com['comment_text'] = com['comment_text'].map(lambda com : remove_long(com))

In [ ]:
categories = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
X = com['comment_text']
y = com[categories]

In [ ]:
X_train , X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
vect = TfidfVectorizer(use_idf=True, ngram_range=(1,1))
tfidf = vect.fit(X_train)
X_train = tfidf.transform(X_train)
X_test = tfidf.transform(X_test)

In [ ]:
pickle.dump(tfidf, open('tfidf.pkl','wb'))

In [ ]:
clf = XGBClassifier(n_estimators=150, n_jobs=-1)
for category in categories:
    print('{}: '.format(category))
    clf.fit(X_train, y_train[category])
    prediction = clf.predict(X_test)
    print('F1 score is {}'.format(f1_score(y_test[category], prediction)))

toxic: 
F1 score is 0.6192090395480226
severe_toxic: 
F1 score is 0.22082018927444796
obscene: 
F1 score is 0.7275641025641024
threat: 
F1 score is 0.2261904761904762
insult: 
F1 score is 0.5829619921363041
identity_hate: 
F1 score is 0.36092715231788075


In [ ]:
clf = LogisticRegression(n_jobs=-1, class_weight='balanced')
for category in categories:
    print('{}: '.format(category))
    clf.fit(X_train, y_train[category])
    prediction = clf.predict(X_test)
    print('F1 score is {}'.format(f1_score(y_test[category], prediction)))

toxic: 
F1 score is 0.7433869648213799
severe_toxic: 
F1 score is 0.41297935103244837
obscene: 
F1 score is 0.7805040894675348
threat: 
F1 score is 0.2696629213483146
insult: 
F1 score is 0.6711047709621176
identity_hate: 
F1 score is 0.36412749864937877


In [ ]:
clf = RandomForestClassifier(max_depth=None, n_estimators=70)
for category in categories:
    print('{}: '.format(category))
    clf.fit(X_train, y_train[category])
    prediction = clf.predict(X_test)
    print('F1 score is {}'.format(f1_score(y_test[category], prediction)))

In [ ]:
clf = clf = MultinomialNB()
for category in categories:
    print('{}: '.format(category))
    clf.fit(X_train, y_train[category])
    prediction = clf.predict(X_test)
    print('F1 score is {}'.format(f1_score(y_test[category], prediction)))

toxic: 
F1 score is 0.33812193412754027
severe_toxic: 
F1 score is 0.0
obscene: 
F1 score is 0.24088323854131816
threat: 
F1 score is 0.014084507042253521
insult: 
F1 score is 0.1203065134099617
identity_hate: 
F1 score is 0.0


# Creating models for each label

In [ ]:
model = LogisticRegression(n_jobs=-1, class_weight='balanced')

In [ ]:
model.fit(X_train, y_train[categories[0]])
pickle.dump(model, open('toxic.pkl','wb'))

In [ ]:
model.fit(X_train, y_train[categories[1]])
pickle.dump(model, open('severe_toxic.pkl','wb'))

In [ ]:
model.fit(X_train, y_train[categories[2]])
pickle.dump(model, open('obscene.pkl','wb'))

In [ ]:
model.fit(X_train, y_train[categories[3]])
pickle.dump(model, open('threat.pkl','wb'))

In [ ]:
model.fit(X_train, y_train[categories[4]])
pickle.dump(model, open('insult.pkl','wb'))

In [ ]:
model.fit(X_train, y_train[categories[5]])
pickle.dump(model, open('identity_hate.pkl','wb'))

In [ ]:
pip install flask_ngrok

In [ ]:
from flask import Flask, request, render_template
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)

def text_preprocess(sent):
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer() 
    
    # convert all words to lowercase
    sent = sent.lower()
    
    # expand contractions
    expanded_words = []
    for word in sent.split():
        if len(word)>20:
            continue
        else:
            expanded_words.append(contractions.fix(word))
    sent = ' '.join(expanded_words) 
    
    # remove html tags
    sent = re.sub('{html}', "", sent)
    
    # remove http links and web site url
    sent = re.sub(r"http\S+", "", sent)
    sent = re.sub(r"www\S+", "", sent)
    
    # remove numbers
    sent = re.sub('[0-9]+', '', sent)
    
    # remove words utc and image
    sent = sent.replace('utc', '')
    
    # remove words with .jpg
    sent = re.sub(r"[a-zA-Z]*[0-9]*.jpg", '', sent)
    
    # remove emails
    sent = re.sub(r"\S*@\S*\s?", '', sent)
    
    # tokenization
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sent.lower())
    
    # remove stop words
    filtered_words = [w for w in tokens if w not in stopwords.words('english')]
    
    stem_words = [stemmer.stem(w) for w in filtered_words]
    lemma_words = [lemmatizer.lemmatize(w) for w in stem_words]
    
    return " ".join(lemma_words)
 
with open('tfidf.pkl', "rb") as f:
    tfidf = pickle.load(f)

with open('toxic.pkl', "rb") as f:
    toxic = pickle.load(f)
with open('severe_toxic.pkl', "rb") as f:
    severe_toxic = pickle.load(f)
with open('obscene.pkl', "rb") as f:
    obscene = pickle.load(f)
with open('threat.pkl', "rb") as f:
    threat = pickle.load(f)
with open('insult.pkl', "rb") as f:
    insult = pickle.load(f)
with open('identity_hate.pkl', "rb") as f:
    identity_hate = pickle.load(f)


@app.route('/')
def home():
   return render_template('index.html')
            
@app.route('/predict', methods=['POST'])
def predict():
   comment = [text_preprocess(str(x)) for x in request.form.values()]
   data = tfidf.transform(comment)
   toxic_pred = toxic.predict_proba(data)
   severe_toxic_pred = severe_toxic.predict_proba(data)
   obscene_pred = obscene.predict_proba(data)
   threat_pred = threat.predict_proba(data)
   insult_pred = insult.predict_proba(data)
   identity_hate_pred = identity_hate.predict_proba(data)
   
   #labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
   #results = [toxic_pred[0][1], severe_toxic_pred[0][1], obscene_pred[0][1], threat_pred[0][1], insult_pred[0][1], identity_hate_pred[0][1]]

   return render_template('index.html', prediction_text="Probabilities are: toxic -> {0}, severe_toxic -> {1}, obscene -> {2}, threat -> {3}, insult -> {4}, identity_hate -> {5}".format(toxic_pred[0][1], 
                                     severe_toxic_pred[0][1],
                                     obscene_pred[0][1],
                                     threat_pred[0][1],
                                     insult_pred[0][1],
                                     identity_hate_pred[0][1]))

if __name__ == "__main__":
   app.run()

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://d81b11e4b716.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [12/Dec/2020 00:42:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2020 00:42:21] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [12/Dec/2020 00:42:26] "POST /predict HTTP/1.1" 200 -
